In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import matlab.engine
import pathlib
from scipy.stats import expon, skew, kurtosis, gaussian_kde
from scipy.stats import pearsonr
import json

In [2]:
eng = matlab.engine.start_matlab()

In [ ]:
#eng.quit()

In [3]:
proj_root = pathlib.Path("/Users/jmoo2880/Documents/hctsa")
eng.addpath(eng.genpath(str(proj_root)), nargout=0)

In [4]:
dset_names = pd.read_csv("../../empirical1000/hctsa_timeseries-info.csv")
dset_names.head(5)

,ID,Name,Keywords,Length
0,1,NS_beta_L10000_a1_b3_2.dat,"synthetic,noise,beta",10000
1,2,NS_beta_L10000_a4_b10_1.dat,"synthetic,noise,beta",10000
2,3,NS_beta_L10000_a4_b3_2.dat,"synthetic,noise,beta",10000
3,4,NS_beta_L1000_a3_b5_2.dat,"synthetic,noise,beta",1000
4,5,NS_beta_L1000_a4_b5_2.dat,"synthetic,noise,beta",1000


In [103]:
pwd()

'/Users/jmoo2880/Documents/py-hctsa-project/pyhctsa'

In [102]:
empirical1000 = []
with open("../../empirical1000/hctsa_timeseries-data.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        # Convert each element from string to float (or int if appropriate)
        try:
            time_series = [float(value) for value in row if value != '']
            empirical1000.append(time_series)
        except ValueError as e:
            print(f"Skipping row due to conversion error: {row}")
            continue

In [6]:
def nan_aware_relative_error(a, b):
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)

    # Masks
    both_nan = np.isnan(a) & np.isnan(b)
    one_nan = np.isnan(a) ^ np.isnan(b)
    neither_nan = ~np.isnan(a) & ~np.isnan(b)

    rel_error = np.empty_like(a, dtype=float)
    
    # Handle NaNs
    rel_error[both_nan] = 0.0
    rel_error[one_nan] = np.inf

    # Avoid division by zero
    safe_a = np.where(np.abs(a) < 1e-12, np.nan, a)  # define "zero" threshold if needed

    # Compute relative error where defined
    rel_error[neither_nan] = np.abs(a[neither_nan] - b[neither_nan]) / np.abs(safe_a[neither_nan])

    # Where a is zero, fall back to using b if possible
    fallback = (neither_nan) & (np.abs(a) < 1e-12) & (np.abs(b) > 1e-12)
    rel_error[fallback] = np.abs(a[fallback] - b[fallback]) / np.abs(b[fallback])

    # Still zero in both a and b → no error
    both_zero = (a == 0) & (b == 0)
    rel_error[both_zero] = 0.0

    return rel_error

In [7]:
from Operations_OLD.CO import StickAngles, BF_zscore, GLSCF, RM_AMInformation, FirstCrossing, PartialAutoCorr

In [8]:
from Operations.Medical import RawHRVMeas, HRV_Classic, PolVar, PNN

In [13]:
PolVar((empirical1000[0]), 1, 3)

0.33323332333233324

In [41]:
eng.MD_polvar(matlab.double(empirical1000[1]), matlab.double(1), matlab.double(3))

0.33323332333233324

In [79]:
nan_aware_relative_error(p1, p2)

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.97156341e-12, 1.13784056e-15, 1.29939953e-15,
       1.97301510e-12, 1.08566193e-01, 4.59051871e-16, 0.00000000e+00])

In [64]:
errs = []
corrs = []
for i in range(len(empirical1000)):
    ml_out = np.array(list(eng.MD_hrv_classic(matlab.double(BF_zscore(empirical1000[i]))).values()))
    python_out = np.array(list(HRV_classic(BF_zscore(empirical1000[i])).values()))
    err = nan_aware_abs_error(python_out, ml_out)
    corr = pearsonr(python_out, ml_out).statistic
    corrs.append(corr)
    errs.append(err)
print(np.max(errs))

2.1715940262115208


In [82]:
res = dict()
for i in range(len(empirical1000)):
    ml_out = np.array(list(eng.MD_hrv_classic(matlab.double(BF_zscore(empirical1000[i]))).values()))
    python_out = np.array(list(HRV_classic(BF_zscore(empirical1000[i])).values()))
    err = nan_aware_relative_error(python_out, ml_out)
    corr = pearsonr(python_out, ml_out)
    res[i] = {"matlab": ml_out, "python" : python_out, "rel_err": err, "pearsonr": corr.statistic, "pval": corr.pvalue} 
#res_modified = {"detailed": res, "summary_stats": {"max err": {np.max()}}}

In [41]:
res = dict()
for i in range(len(empirical1000)):
    ml_out = np.array(list(eng.MD_rawHRVmeas(matlab.double((empirical1000[i]))).values()))
    python_out = np.array(list(RawHRVMeas((empirical1000[i])).values()))
    err = nan_aware_relative_error(python_out, ml_out)
    corr = pearsonr(python_out, ml_out)
    res[i] = {"matlab": ml_out, "python" : python_out, "rel_err": err, "pearsonr": corr.statistic, "pval": corr.pvalue} 
np.mean([res[i]['pearsonr'] for i in res])

np.float64(0.9992053726594734)

In [22]:
res = dict()
for i in range(len(empirical1000)):
    ml_out = np.array(list(eng.MD_pNN(matlab.double((empirical1000[i]))).values()))
    python_out = np.array(list(PNN((empirical1000[i])).values()))
    err = nan_aware_relative_error(python_out, ml_out)
    corr = pearsonr(python_out, ml_out)
    res[i] = {"matlab": ml_out, "python" : python_out, "rel_err": err, "pearsonr": corr.statistic, "pval": corr.pvalue} 
np.nanmean([res[i]['pearsonr'] for i in res])

/var/folders/pt/9v0934pn6s1g4klqjbcz65p40000gp/T/ipykernel_73497/582295190.py:6: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr = pearsonr(python_out, ml_out)


np.float64(1.0)

In [ ]:
eng.MD_polvar(matlab.double(empirical1000[1]), matlab.double(1), matlab.double(3))

0.33323332333233324

In [95]:
res = dict()
i1, i2 = 0.1, 6
for i in range(len(empirical1000)):
    ml_out = eng.MD_polvar(matlab.double(empirical1000[i]), matlab.double(i1), matlab.double(i2))
    python_out = PolVar((empirical1000[i]), i1, i2)
    err = nan_aware_relative_error(python_out, ml_out)
    #corr = pearsonr(python_out, ml_out)
    res[i] = {"matlab": ml_out, "python" : python_out, "rel_err": err} 
#np.nanmean([res[i]['pearsonr'] for i in res])

In [96]:
pearsonr([res[i]['matlab'] for i in res], [res[i]['python'] for i in res])

PearsonRResult(statistic=np.float64(0.999889442946935), pvalue=np.float64(0.0))

In [97]:
np.mean(res[i]['rel_err'])

np.float64(0.0)

In [ ]:
# with open('md_.json', 'w') as f:
#     json.dump(res, f)

{0: {'matlab': array([9.95899590e-01, 9.92199220e-01, 9.85698570e-01, 9.78497850e-01,
         9.71897190e-01, 4.72374622e-01, 1.30919139e+00, 3.88411190e+00,
         8.22252450e+00, 3.94166338e+00, 9.95063424e+02, 1.00491233e+03]),
  'python': array([9.95899590e-01, 9.92199220e-01, 9.85698570e-01, 9.78497850e-01,
         9.71897190e-01, 4.72374622e-01, 1.30919139e+00, 3.88411190e+00,
         8.22252450e+00, 3.94166338e+00, 9.95063424e+02, 1.00491233e+03]),
  'abs_err': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  'pearsonr': np.float64(1.0),
  'pval': np.float64(0.0)},
 1: {'matlab': array([9.97699770e-01, 9.95099510e-01, 9.89999000e-01, 9.85298530e-01,
         9.78897890e-01, 5.11507764e-01, 1.31825830e+00, 3.81163997e+00,
         7.45177343e+00, 4.08163265e+00, 1.00352416e+03, 9.96463375e+02]),
  'python': array([9.97699770e-01, 9.95099510e-01, 9.89999000e-01, 9.85298530e-01,
         9.78897890e-01, 5.11507764e-01, 1.31825830e+00, 3.81163997e+00,
         7.45177